In [ ]:
# Author: Viraaj Minhas
from tkinter import *
import random
from tkinter import messagebox
class MineSweeperCell(Label):
    '''represents one cell in a mine sweeper grid'''

    def __init__(self, master, coord):
        '''MineSweeperCell(master, coord) -> MineSweeperCell
        creates a MineSweeperCell object with at an inputted coordinate'''

        #Initializing superclass
        Label.__init__(self, master, height = 2, width = 4, bg = "white", relief = "groove", font = ("Arial", 14))

        #Initializing identifying attributes of a MineSweeperCell object
        self.coord = coord
        self.bombNeighbours = 0
        self.flagged = False
        self.exposed = False
        self.isBomb = False
        self.hasWon = False

        #Left mouse button exposes a cell and right mouse button flaggs a cell
        self.bind("<Button-1>", self.expose)
        self.bind("<Button-3>", self.flag)

    def is_bomb(self):
        '''MineSweeperCell.is_bomb() -> bool
        returns whether a cell is a bomb cell'''
        return self.isBomb

    def make_bomb(self):
        '''MineSweeperCell.make_bomb()
        assigns a cells isBomb attribute to be True in order to make it a bomb'''
        self.isBomb = True

    def set_bomb_neighbours(self, number):
        '''MineSweeperCell.set_bomb_neighbours()
        sets the number of a cell to a specific number'''
        self.bombNeighbours = number

    def highlight(self):
        '''MineSweeperCell.highlight()
        makes a cell's background grey and exposes it'''

        #You can only highlight squares that are not already exposed or flagged
        if not self.exposed and not self.flagged:
            self["bg"] = "lightgrey"

            #Exposing the square
            self.exposed = True


    def update_display(self):
        '''MineSweeperCell.update_display()
        updates the text and color of a cell'''

        #If it has no bomb neighbours and is not a bomb
        if self.bombNeighbours == 0 and not self.isBomb:

            #Displays an empty box
            self["text"] = ""

        #Only change the color and text of cells that are not bombs and are not flagged
        elif not self.flagged and not self.isBomb:
            self["text"] = self.bombNeighbours
            colormap = ['','blue','darkgreen','red','purple','maroon','cyan','black','dim gray']
            self["fg"] = colormap[self.bombNeighbours]

    def has_won(self):
        '''MineSweeperCell.has_won()
        determines if a player has won and carries out all necessary actions
        if the player has won, it exposes all bombs, deactivates all cells, and displays a messagebox'''

        #Initializing how many cells remain unexposed
        cellsRemaining = 0

        #If the game is still going on
        if not self.hasWon:

            #Checking all cells
            for cell in self.master.get_cells():

                #Determining how many cells have not been interacted with yet
                if not cell.exposed and not cell.flagged:

                    #It still needs to be clicked in order for the player to win
                    cellsRemaining += 1

        #If the player has clicked on the last cell (the player has won)
        if cellsRemaining == 0:

            #Highlight the cell the player just clicked on
            #self.highlight()

            for cell in self.master.get_cells():

                #Updating all cells and deactivating them (end of game)
                #cell.highlight()

                #cell.update_display()
                cell.unbind("<Button-1>")
                cell.unbind("<Button-3>")

            #Tell the player that they have won
            messagebox.showinfo('Minesweeper','Congratulations -- you won!',parent=self)
            self.hasWon = True

    def expose(self, event):
        '''MineSweeperCell.expose(event)
        exposes a cell and surrouding cells if necessary
        checks on every click if the player has won and runs the has_won function'''

        if not self.hasWon:    #Prevents this from running if the player already won

            #Check if player has won. If they won, the game ends.
            self.has_won()

        #Only squares that are not exposed, bombs, or are flagged can be highlighted
        if not self.exposed and not self.isBomb and not self.flagged:
            self["relief"] = "sunken"
            self.highlight()

            #Auto-expose feature
            if self.bombNeighbours == 0 and not self.hasWon:

                #Recursively exposing all neighbours
                for neighbour in self.master.get_valid_neighbours(self.coord):
                    neighbour.expose(event)

            #Displaying the changes made to the cells
            self.update_display()

        #The player has lost the game if they expose a bomb
        elif self.isBomb:
            self["relief"] = "sunken"

            #Exposing all bombs
            for cell in self.master.get_cells():
                if cell.isBomb and not cell.flagged:

                    #Making them have an asterisk and a red background
                    cell["text"] = "*"
                    cell["bg"] = "red"
                    cell["relief"] = "sunken"

                    #Updating cells
                    cell.exposed = True
                    cell.update_display()

                #Deactivating all cells
                cell.unbind("<Button-1>")
                cell.unbind("<Button-3>")

            #Alerting the player that they have lost the game
            messagebox.showerror('Minesweeper','KABOOM! You lose.',parent=self.master)


    def flag(self, coord):
        '''MineSweeperCell.flag(coord)
        flags a cell by placing an asterisk on it
        flagged cells can not be exposed until unflagged
        the player wins if all cells except bombs are exposed or flagged'''



        #Only flag if a sell is not already exposed or flagged
        if not self.exposed and not self.flagged:

            #If you have not already used all 15 flags
            if self.master.get_flags_remaining() > 0:

                #Decrease the flag count by 1
                self.master.increase_flags(-1)

                self.flagged = True    #Mark the cell as flagged

                #Make the cell display an asterisk
                self["text"] = "*"

                #Update flag label
                self.master.get_flag_label()["text"] = self.master.get_flags_remaining()

        #If the player is trying to unflag a square
        elif self.flagged:

            #Increase the flag count by one
            self.master.increase_flags(1)

            self.flagged = False    #Mark the cell as unflagged

            #Make the cell empty and update flag label
            self["text"] = ""
            self.master.get_flag_label()["text"] = self.master.get_flags_remaining()

        if not self.hasWon:    #Prevents this from running if the player has already won

            #Check if player has won. If they won, the game ends.
            self.has_won()

class MineSweeperUnit:
    '''A unit containing all neighbouring cells of an inputted focus cell'''

    def __init__(self, cells, focusCell):
        '''MineSweeperUnit(cells, focusCell)
        a unit that contains all the neighbouring cells to the focus cell whithin a MineSweeperGrid object'''

        #Initializing cells and focusCell attributes
        self.cells = cells
        self.focusCell = focusCell

    def num_bomb_neighbours(self, focusCell):
        '''MineSweeperUnit.num_bomb_neighbours -> int
        returns the number of cells in a unit that are bombs'''
        neighbours = self.cells

        #Iterating all neighbours in order to find all bombs
        bombNeighbours = 0
        for neighbour in neighbours:
            if neighbour.is_bomb():
                bombNeighbours += 1

        return bombNeighbours

class MineSweeperGrid(Frame):
    '''a grid containing MineSweeperCells with a width, height, and a certain number of bombs'''

    def __init__(self, master, width, height, numBombs):
        '''MineSweeperGrid(master, width, height, numBombs) -> MineSweeperGrid
        a grid that has cells, a width, a height, and a specific number of bombs'''

        #Initializing superclass
        Frame.__init__(self, master, bg = "black")
        self.grid()

        #Attributres representing input values
        self.numBombs = numBombs
        self.width = width
        self.height = height
        self.flagsRemaining = 30

        #Creating a flag label that displays how many flags the player has to place
        self.flagLabel = Label(self, text = str(self.flagsRemaining), font = ("Arial", 18))
        self.flagLabel.grid(row = self.height, column = int((self.width-1)/2), columnspan = 2)

        #Dictionary of cells in the grid
        self.cells = {}

        #Randomly selecting bomb cells from the grid
        bombCells = random.sample(range(1, self.height * self.width + 1), self.numBombs)

        #Starting one position before the first cell
        cellNumber = 0

        #Iterating over columns and rows
        for row in range(self.height):
            for column in range(self.width):

                #Adding the keys and values of each cell to the cells dictionary
                coord = (row, column)
                self.cells[coord] = MineSweeperCell(self, coord)

                #Displaying the cells
                self.cells[coord].grid(row = row, column = column)

                #Moving on to the next cell
                cellNumber += 1

                if cellNumber in bombCells:
                    self.cells[coord].make_bomb()    #Making these cells bombs

        #Creating dictionaries for each cells valid neighbours and each cells units
        self.validNeighbours = {}
        self.units = {}

        #Iterating over every sell
        for focusCell in self.cells:

            #Initializing each cells value in the validNeighbours dictionary
            self.validNeighbours[focusCell] = []

            #Finding all neighbours of the focusCell
            row, column = focusCell[0], focusCell[1]
            neighbours = [(row - 1, column - 1), (row - 1, column), (row - 1, column + 1), (row, column - 1), (row, column + 1), (row + 1, column - 1), (row + 1, column), (row + 1, column + 1)]

            for neighbour in neighbours:

                #Only selecting neighbours that are whithin the boundary of the grid
                if neighbour[0] > -1 and neighbour[1] > -1 and neighbour[0] < self.height and neighbour[1] < self.width:
                    self.validNeighbours[focusCell].append(self.cells[neighbour])

            #Creating units for each cell and finding the number of neighbouring cells that are bombs
            self.units[focusCell] = MineSweeperUnit(self.validNeighbours[focusCell], focusCell)
            self.cells[focusCell].set_bomb_neighbours(self.units[focusCell].num_bomb_neighbours(self.units[focusCell]))

    def get_cells(self):
        '''MineSweeperGrid.get_cells() -> list
        returns a list containing all of the cells whithin the grid'''
        return list(self.cells.values())

    def get_valid_neighbours(self, coord):
        '''MineSweeperGrid.get_valid_neighbours(coord) -> list
        returns a list containing all of the neighbouring cells of a cell with a given coordinate'''
        return self.validNeighbours[coord]

    def get_flags_remaining(self):
        '''MineSweeperGrid.get_flags_remaining() -> int
        returns an integer representing the number of remaining flags that can be placed'''
        return self.flagsRemaining

    def increase_flags(self, amt):
        '''MineSweeperGrid.increase_flags(amt)
        increases the flagsRemaining attribute by a certain amount'''
        self.flagsRemaining += amt

    def get_flag_label(self):
        '''MineSweeperGrid.get_flag_label()
        returns the label representing the number of flags remaining to be placed'''
        return self.flagLabel

def play_minesweeper(width, height, numBombs):
    '''play_minesweeper(width, height, numBombs
    simulates playing a tkinter version of minesweeper'''

    #Creating a window
    root = Tk()
    root.title('Minesweeper')

    #Initializing the grid
    mg = MineSweeperGrid(root, width, height, numBombs)

    #Listening for inputs
    root.mainloop()

#Plays a game of minesweeper
play_minesweeper(12,10,30)

In [ ]:
from tkinter import *
from tkinter import messagebox
import random

class MinesweeperCell(Label):
    '''Represents one singular cell in the whole gameboard'''
    def __init__(self, master, coord):
        self.master = master
        self.coord = coord
        self.number = 0  # 0 represents an empty cell
        self.highlighted = False  # starts unhighlighted
        self.flagged = False # starts unflagged
        self.disabled = False
        self.is_bomb = False
        self.game_over = False
        self.clicked_cells = set()

        Label.__init__(self, master, height=1, width=2, text='', bg='white', font=('Arial', 24))

        self.bind('<Button-1>', self.highlight_cell)
        self.bind('<Button-3>', self.flag_cell)

    def highlight_cell(self, event):
        '''MinesweeperCell.highlight(event)
        handler function for mouse click
        highlights the cell and displays the adjacent bombs'''

        if self.game_over == True: # Check if the game is over
            self.game_outcome() # Don't do anything if the game is over

        # if self.disabled == True: # If already disabled, do nothing
            # return

        # Proceed with highlighting the cell
        self.focus_set()  # set the focus so we can capture key presses
        self.bind('<Button-1>', self.highlight_cell)
        self.highlighted = True
        self.disabled = True
        self['bg'] = 'lightgrey'
        self.clicked_cells.add(self.coord)
        self.num_of_adjacent_bombs(self.coord)
        self.unbind('<Button-3>')  # Unbind right-click so no flagging happens

        if self.is_bomb == True: # If it's a bomb, immediately end the game
            self.unbind('<Button-1>')
            self.clicked_cells.add(self.coord)
            self.game_outcome()

    def flag_cell(self, event):

        if self.game_over == True:  # Check if the game is over
            self.game_outcome()  # Don't do anything if the game is over

        if self.highlighted == True:  # If the cell is already highlighted, prevent flagging
            return

        if self.flagged == True:
            self.focus_set()
            self.bind('<Button-3>', self.flag_cell)
            self.flagged = False
            self.config(text='')
            self.highlighted = False
            self.bind('<Button-1>', self.highlight_cell)

        elif self.flagged == False:
            self.focus_set()
            self.bind('<Button-3>', self.flag_cell)
            self.flagged = True
            self.clicked_cells.add(self.coord)
            self.config(text='*')
            self.unbind('<Button-1>')

    def num_of_adjacent_bombs(self, coord):
        colormap = ['', 'blue', 'darkgreen', 'red', 'purple', 'maroon', 'cyan', 'black', 'dim gray']
        adjacent_bombs = set()  # Initialize counter

        row, col = coord

        # Check adjacent rows
        if (row - 1, col) in MinesweeperGrid.return_cells(self.master) and MinesweeperGrid.return_cells(self.master)[(row - 1, col)].is_bomb:
            adjacent_bombs.add((row - 1, col))

        if (row + 1, col) in MinesweeperGrid.return_cells(self.master) and MinesweeperGrid.return_cells(self.master)[(row + 1, col)].is_bomb:
            adjacent_bombs.add((row + 1, col))

        # Check adjacent columns
        if (row, col + 1) in MinesweeperGrid.return_cells(self.master) and MinesweeperGrid.return_cells(self.master)[(row, col + 1)].is_bomb:
            adjacent_bombs.add((row, col + 1))

        if (row, col - 1) in MinesweeperGrid.return_cells(self.master) and MinesweeperGrid.return_cells(self.master)[(row, col - 1)].is_bomb:
            adjacent_bombs.add((row, col - 1))

        # Check adjacent rows
        if (row - 1, col) in MinesweeperGrid.return_cells(self.master) and MinesweeperGrid.return_cells(self.master)[(row - 1, col)].is_bomb:
            adjacent_bombs.add((row - 1, col))

        if (row + 1, col) in MinesweeperGrid.return_cells(self.master) and MinesweeperGrid.return_cells(self.master)[(row + 1, col)].is_bomb:
            adjacent_bombs.add((row + 1, col))

        # Check all four diagonals
        if (row - 1, col + 1) in MinesweeperGrid.return_cells(self.master) and MinesweeperGrid.return_cells(self.master)[(row - 1, col + 1)].is_bomb:
            adjacent_bombs.add((row - 1, col + 1))

        if (row - 1, col - 1) in MinesweeperGrid.return_cells(self.master) and MinesweeperGrid.return_cells(self.master)[(row - 1, col - 1)].is_bomb:
            adjacent_bombs.add((row - 1, col - 1))

        if (row + 1, col - 1) in MinesweeperGrid.return_cells(self.master) and MinesweeperGrid.return_cells(self.master)[(row + 1, col - 1)].is_bomb:
            adjacent_bombs.add((row + 1, col - 1))

        if (row + 1, col + 1) in MinesweeperGrid.return_cells(self.master) and MinesweeperGrid.return_cells(self.master)[(row + 1, col + 1)].is_bomb:
            adjacent_bombs.add((row + 1, col + 1))


        # Get the length of the set and the respective colored number
        length_of_set = len(adjacent_bombs)

        if length_of_set > 0:
            color = colormap[length_of_set]
            return self.config(text=str(length_of_set), fg=str(color))
        else:
            pass

    def return_clicked_cells(self):
        return self.clicked_cells

    def game_outcome(self):
        for cell in self.clicked_cells:
            if cell in MinesweeperGrid.return_bomb_list(self.master):
                self.game_over = True
                messagebox.showerror('Minesweeper', 'KABOOM! You lose.', parent=self)
                self.master.master.destroy()

        if MinesweeperGrid.return_safe_squares(self.master) == len(self.clicked_cells):
            self.game_over = True
            messagebox.showinfo('Minesweeper', 'Congratulations -- you won!', parent=self)
            self.master.master.destroy()


class MinesweeperGrid(Frame):
    '''represents the whole game grid'''
    def __init__(self, master, width, height, numBombs):
        self.master = master
        self.width = width
        self.height = height
        self.numBombs = numBombs
        self.bombList = []
        self.is_bomb = False

        Frame.__init__(self, master, bg='black', width = 500, height = 500)
        self.grid()

        for n in range(1, 2 * int(self.height), 2):
            self.rowconfigure(n, minsize=1)
        for n in range(1, 2 * int(self.width), 2):
            self.columnconfigure(n, minsize=1)

        self.cells = {}  # set up dictionary for cells
        for row in range(int(self.height)):
            for column in range(int(self.width)):
                coord = (row, column)
                self.cells[coord] = MinesweeperCell(self, coord)
                # cells go in even-numbered rows/columns of the grid
                self.cells[coord].grid(row=2 * row, column=2 * column)

        # Randomly selecting bombs and assigning bomb attribute
        self.bombList = random.sample(list(self.cells), numBombs)

        for bomb_coord in self.bombList:
            if bomb_coord in self.cells:
                self.cells[bomb_coord].is_bomb = True

            self.cells[bomb_coord].config(bg='red')

    def return_cells(self):
        return self.cells

    def return_is_bomb(self):
        return self.is_bomb

    def return_bomb_list(self):
        return self.bombList

    def return_safe_squares(self):
        return int(self.width * self.height) - int(self.numBombs)


def play_minesweeper():
    root = Tk()
    root.title("Minesweeper")
    MinesweeperGrid(root, 5, 5, 10)
    root.mainloop()

play_minesweeper()